In [2]:
import pandas as pd

listings = pd.read_csv('listings.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,...,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11
2,8249488,https://www.airbnb.com/rooms/8249488,20151002231825,2015-10-03,Spacious/private room for single,This is an ideal room for a single traveler th...,NaN,This is an ideal room for a single traveler th...,none,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,1.00
3,8409022,https://www.airbnb.com/rooms/8409022,20151002231825,2015-10-03,A wonderful bedroom with library,Prime location right on the Potomac River in W...,NaN,Prime location right on the Potomac River in W...,none,NaN,...,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,1,NaN
4,8411173,https://www.airbnb.com/rooms/8411173,20151002231825,2015-10-03,Downtown Silver Spring,"Hi travellers! I live in this peaceful spot, b...",This is a 750 sq ft 1 bedroom 1 bathroom. Whi...,"Hi travellers! I live in this peaceful spot, b...",none,Silver Spring is booming. You can walk to a n...,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


In [3]:
dc_listings = listings.loc[:,['host_response_rate','host_acceptance_rate',
                              'host_listings_count','accommodates','room_type',
                             'bedrooms','bathrooms','beds','price','cleaning_fee',
                              'security_deposit','minimum_nights','maximum_nights',
                             'number_of_reviews','latitude','longitude','city','zipcode','state']]
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
0,92%,91%,26,4,Entire home/apt,1.0,1.0,2.0,$160.00,$115.00,$100.00,1,1125,0,38.890046,-77.002808,Washington,20003,DC
1,90%,100%,1,6,Entire home/apt,3.0,3.0,3.0,$350.00,$100.00,NaN,2,30,65,38.880413,-76.990485,Washington,20003,DC
2,90%,100%,2,1,Private room,1.0,2.0,1.0,$50.00,NaN,NaN,2,1125,1,38.955291,-76.986006,Hyattsville,20782,MD
3,100%,NaN,1,2,Private room,1.0,1.0,1.0,$95.00,NaN,NaN,1,1125,0,38.872134,-77.019639,Washington,20024,DC
4,92%,67%,1,4,Entire home/apt,1.0,1.0,1.0,$50.00,$15.00,$450.00,7,1125,0,38.996382,-77.041541,Silver Spring,20910,MD


In [4]:
#The K-nearest neighbors algorithm
"""
            Euclidian Distance
d = \sqrt{(q_1-p_1)^2 + (q_2-p_2)^2 + \cdots + (q_n-p_n)^2}
q_1 to q_n represent the feature values for one observation
p_1 to p_n represent the feature values for the other observation


Let's look at just one column from our data set
d = \sqrt{(q_1 – p_1)^2}
d = | q_1 – p_1 |
we want to find the absolute value of the difference between the observation 
and the data point we want to predict for the feature we’re using


let’s say the living space that we want to rent can accommodate three people
we’ll first calculate the distance, using just the accommodates feature, 
between the first living space in the data set and our own.
"""

import numpy as np
our_acc_value = 3
first_living_space_value = dc_listings.loc[0,'accommodates']
first_living_space_value

4

In [6]:
first_distance = np.abs(first_living_space_value - our_acc_value)
first_distance

1

In [16]:
"""
The smallest possible Euclidian distance is zero, which would mean the observation we are comparing to is identical to ours, 
so the result we’ve gotten here makes sense. 
In isolation, however, the value doesn’t mean much unless we know how it compares to other values. 
Let’s calculate the Euclidean distance for each observation in our data set
"""
dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [19]:
"""
There are 461 listings that have a distance of 0, and thus accommodate the same number of people as our listing. 
Those listings might be a good place to start.
If we just used the first five values with a distance of 0, our predictions would be biased to the existing ordering of the data set. 
Instead, we’ll randomize the ordering of the observations and then select the first five rows with a distance of 0

We’ll also use the random_state parameter which just gives us a reproducible random order 
so anyone can follow along and get the exact same results.
"""
dc_listings = dc_listings.sample(frac=1,random_state=0)
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
2177,90%,100%,5,2,Private room,1.0,1.0,1.0,$80.00,NaN,NaN,1,1125,2,38.904929,-77.008539,Washington,20002,DC,1
2053,100%,95%,5,6,Entire home/apt,3.0,2.5,4.0,$679.00,$100.00,$500.00,3,1125,2,38.898319,-76.987931,Washington,20002,DC,3
2920,100%,100%,1,2,Private room,1.0,1.0,1.0,$50.00,NaN,NaN,1,1125,0,38.928904,-77.028278,Washington,20010,DC,1
2919,84%,100%,1,3,Entire home/apt,1.0,1.0,1.0,$150.00,$50.00,$100.00,1,1125,1,38.925646,-77.033456,Washington,20009,DC,0
695,100%,86%,1,2,Private room,1.0,1.0,1.0,$110.00,NaN,NaN,1,30,157,38.906686,-77.033052,Washington,20005,DC,1


In [22]:
dc_listings = dc_listings.sort_values('distance')
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
740,80%,0%,1,3,Entire home/apt,1.0,1.0,1.0,$165.00,$25.00,NaN,1,1125,4,38.917935,-77.096023,Washington,20007,DC,0
3169,NaN,0%,1,3,Entire home/apt,2.0,2.5,1.0,$139.00,$75.00,NaN,2,1125,5,38.926106,-77.033078,Washington,20009,DC,0
453,92%,76%,206,3,Entire home/apt,1.0,1.0,2.0,$207.00,$100.00,NaN,3,1125,2,38.909272,-77.032468,Washington,20005,DC,0
602,75%,100%,1,3,Entire home/apt,1.0,1.0,1.0,$130.00,NaN,$150.00,2,1125,2,38.910553,-77.023300,Washington,20001,DC,0
2348,100%,100%,1,3,Entire home/apt,0.0,1.0,2.0,$138.00,NaN,NaN,2,150,68,38.920233,-77.042801,Washington,20009,DC,0


In [23]:
dc_listings.price.head()

740     $165.00
3169    $139.00
453     $207.00
602     $130.00
2348    $138.00
Name: price, dtype: object

In [28]:
"""
Before we can take the average of our prices, you’ll notice that our price column has the object type, 
due to the fact that the prices have dollar signs and commas
"""
dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)
dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance
740,80%,0%,1,3,Entire home/apt,1.0,1.0,1.0,165.0,$25.00,NaN,1,1125,4,38.917935,-77.096023,Washington,20007,DC,0
3169,NaN,0%,1,3,Entire home/apt,2.0,2.5,1.0,139.0,$75.00,NaN,2,1125,5,38.926106,-77.033078,Washington,20009,DC,0
453,92%,76%,206,3,Entire home/apt,1.0,1.0,2.0,207.0,$100.00,NaN,3,1125,2,38.909272,-77.032468,Washington,20005,DC,0
602,75%,100%,1,3,Entire home/apt,1.0,1.0,1.0,130.0,NaN,$150.00,2,1125,2,38.910553,-77.023300,Washington,20001,DC,0
2348,100%,100%,1,3,Entire home/apt,0.0,1.0,2.0,138.0,NaN,NaN,2,150,68,38.920233,-77.042801,Washington,20009,DC,0


In [55]:
mean_price = dc_listings.price.iloc[:5].mean()
mean_price

155.8

In [50]:
#Evaluating Our Model
"""
Split the data set into 2 partitions:
    A training set: contains the majority of the rows (75%)
    A test set: contains the remaining minority of the rows (25%)

Let's start removing the distance column we added earlier, and spliting the data.
No need to worry about randomizing the rows, since they’re still randomized from earlier
"""
dc_listings.drop('distance', axis=1)
train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return predicted_price

In [59]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price, feature_column='accommodates')

In [60]:
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,...,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance,predicted_price
1653,85%,79%,19,1,Private room,1.0,4.0,1.0,NaN,$15.00,...,4,365,4,38.938150,-77.023779,Washington,20010,DC,2,NaN
769,100%,NaN,1,1,Private room,1.0,1.0,1.0,NaN,NaN,...,1,1125,0,38.900692,-76.974978,Washington,20002,DC,2,NaN
1059,92%,99%,23,1,Private room,1.0,1.5,1.0,NaN,NaN,...,1,1125,6,38.914771,-77.036525,Washington,20009,DC,2,NaN
3288,NaN,NaN,1,5,Entire home/apt,3.0,2.0,3.0,NaN,NaN,...,3,1125,0,38.936027,-76.976049,Washington,20018,DC,2,NaN
3634,75%,100%,1,1,Private room,1.0,1.5,1.0,NaN,NaN,...,3,1125,37,38.891749,-76.990916,Washington,20002,DC,2,NaN


In [61]:
#Using RMSE to Evaluate Our Model
"""
For many prediction tasks, we want to penalize predicted values that are further away from 
the actual value much more than those that are closer to the actual value.

RMSE = \sqrt {\dfrac{ (actual_1-predicted_1)^2 + (actual_2-predicted_2)^2 + \cdots + (actual_n-predicted_n)^2 }{ n }}
"""
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,...,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,distance,predicted_price,squared_error
1653,85%,79%,19,1,Private room,1.0,4.0,1.0,NaN,$15.00,...,365,4,38.938150,-77.023779,Washington,20010,DC,2,NaN,NaN
769,100%,NaN,1,1,Private room,1.0,1.0,1.0,NaN,NaN,...,1125,0,38.900692,-76.974978,Washington,20002,DC,2,NaN,NaN
1059,92%,99%,23,1,Private room,1.0,1.5,1.0,NaN,NaN,...,1125,6,38.914771,-77.036525,Washington,20009,DC,2,NaN,NaN
3288,NaN,NaN,1,5,Entire home/apt,3.0,2.0,3.0,NaN,NaN,...,1125,0,38.936027,-76.976049,Washington,20018,DC,2,NaN,NaN
3634,75%,100%,1,1,Private room,1.0,1.5,1.0,NaN,NaN,...,1125,37,38.891749,-76.990916,Washington,20002,DC,2,NaN,NaN


In [62]:
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

nan

In [63]:
#Comparing different models

for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: nan
RMSE for the bedrooms column: nan
RMSE for the bathrooms column: nan
RMSE for the number_of_reviews column: nan
